In [ ]:
import re, time
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

Tensorflow version 2.3.0


In [ ]:
import tensorflow as tf
import os

try: # detect TPUs
  tpu = None
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError: # detect GPUs
  strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
  #strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines

print("Number of accelerators: ", strategy.num_replicas_in_sync)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of accelerators:  1


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd /content/gdrive/MyDrive/visual_qa

Mounted at /content/gdrive
/content/gdrive/MyDrive/visual_qa


In [ ]:
input_json='data/data_prepro.json'
input_img_h5='data/data_img.h5'
input_ques_h5='data/data_prepro.h5'

In [ ]:
import h5py
data_ques = h5py.File(input_ques_h5,'r')
data_img = h5py.File(input_img_h5,'r')
print(data_ques.keys())
print(data_img.keys())

<KeysViewHDF5 ['MC_ans_test', 'answers', 'img_pos_test', 'img_pos_train', 'ques_length_test', 'ques_length_train', 'ques_test', 'ques_train', 'question_id_test', 'question_id_train']>
<KeysViewHDF5 ['images_test', 'images_train']>


In [ ]:
NUM_TRAIN, QUES_LEN = data_ques["ques_train"].shape
NUM_TEST, QUES_LEN = data_ques["ques_test"].shape
print(NUM_TRAIN, NUM_TEST, QUES_LEN)

215359 121512 26


In [ ]:
NUM_IMGS_TRAIN, IMG_DIM =  data_img["images_train"].shape
NUM_IMGS_TEST, IMG_DIM =  data_img["images_test"].shape
print(NUM_IMGS_TRAIN, NUM_IMGS_TEST, IMG_DIM)

82459 40504 4096


In [ ]:
import json
file = open(input_json)
data_dict = json.load(file)
print(data_dict.keys())

dict_keys(['ix_to_word', 'unique_img_test', 'ix_to_ans', 'unique_img_train'])


In [ ]:
print(data_dict['unique_img_train'][0], len(data_dict['unique_img_train'])) #list
print(data_dict['unique_img_test'][0], len(data_dict['unique_img_test'])) #list
print(data_dict['ix_to_word']) #dict starting with 1
VOCAB_SIZE = len(data_dict['ix_to_word'])
print(data_dict['ix_to_word']['1'], len(data_dict['ix_to_word'])) 
print(data_dict['ix_to_ans']) #dict starting with 1
print(data_dict['ix_to_ans']['1'], len(data_dict['ix_to_ans'])) 

train2014/COCO_train2014_000000362839.jpg 82459
val2014/COCO_val2014_000000533942.jpg 40504
{'1': 'raining', '2': 'writings', '3': 'yellow', '4': 'four', '5': 'prices', '6': 'woods', '7': 'spiders', '8': 'hanging', '9': 'woody', '10': 'he/her', '11': 'aggression', '12': 'marching', '13': 'increase', '14': 'canes', '15': 'electricity', '16': 'similarity', '17': 'sunlit', '18': 'skipping', '19': 'fingernails', '20': 'deckered', '21': 'immature', '22': 'flicking', '23': 'shaving', '24': 'sinking', '25': 'slobs', '26': 'digit', '27': 'shielding', '28': 'button-down', '29': 'deli', '30': 'regional', '31': 'fruit/veg', '32': 'slenderman', '33': 'hdtv', '34': 'right-rear', '35': 'leisurely', '36': 'hibernating', '37': 'fur', '38': 'screaming', '39': 'showroom', '40': 'wooded', '41': 'prize', '42': 'yachting', '43': 'wooden', '44': 'protest', '45': 'crotch', '46': 'shortboard', '47': 'ornamental', '48': 'charter', '49': 'daybed', '50': 'tired', '51': 'railing', '52': 'scrapes', '53': 'tires', 